<a href="https://colab.research.google.com/github/Athugodage/Gazprom_test/blob/main/gazprom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip '/content/кейс - Разработчик NLP.zip'

In [ ]:
!unzip '/content/nlp/journals_shot.zip'

In [ ]:
import pandas as pd

df = pd.read_csv('/content/journals_shot.tsv', sep='\t')

In [ ]:
df

In [ ]:
code2text = pd.read_csv('/content/nlp/codeToText.tsv', sep='\t', header=None)
code2text

Предобработка данных

In [ ]:
d = dict()

for ind in code2text.index:
    code = code2text[0].iloc[ind]
    text = code2text[1].iloc[ind]
    d[code] = text

print(d)

In [ ]:
## создаем столбец с русскоязычными действиями

df['eventText'] = df['eventCode'].apply(lambda x: d[x])

In [ ]:
df['eventTime'] = pd.to_datetime(df['eventTime'])
df['time'] = df['eventTime'].apply(lambda x: x.time().hour)

**Задание 1.**

1) Сколько всего пользователей?

In [ ]:
print('Всего пользователей: ', len(set(df['userId'])))

2) В какое время суток пользователи заходят?

In [ ]:
df[df['eventText'] == 'Просмотрен объект']

In [ ]:
##  смотрим частотату захода по каждому часу

freq = df[df['eventText'] == 'Просмотрен объект'].groupby('time', as_index=False).agg({'id':'count'})

In [ ]:
freq

In [ ]:
import matplotlib.pyplot as plt

x = freq['time']  
y = freq['id']  # times of entry





fig, ax = plt.subplots()
ax.plot(x, y)

plt.xlabel('Часы')
plt.ylabel('Количество просмотров')
plt.title('Частота просматриваемости за сутки')

plt.show()


##  как видно из графика, больше всего заходят (если быть точным, просматривают объект)
##  с 9:00 по 16:00

Сколько действий совершают пользователи?

In [ ]:
events = df.groupby('eventText', as_index=False).agg({'userId': 'count'})


In [ ]:
events

In [ ]:
events.loc[len(events.index)] = ['Другое', events[events['userId'] < 500]['userId'].agg('sum')]

In [ ]:
event_list = events[events['userId'] > 500]['eventText'].to_list()
event_freq = events[events['userId'] > 500]['userId'].to_list()
event_list

In [ ]:
import numpy as np

# Creating autocpt arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return absolute/1000

colors = ( "orange", "cyan", "brown",
          "grey", "indigo", "beige", "pink")

fig, ax = plt.subplots(figsize =(20, 15))
wedges, texts, autotexts = ax.pie(event_freq,
                                  autopct = lambda pct: func(pct, event_freq),
                                  shadow = True,
                                  colors = colors,
                                  startangle = 90,
                                  textprops = dict(color ="black"))

ax.legend(wedges, event_list,
          title ="Действия",
          loc ="center left",
          bbox_to_anchor =(1, 0, 0.5, 1))

ax.set_title("Кол-во (в тысячах) каждого действия из всех совершенных")
 
plt.show()

Задание 2.

1) Что ищут пользователи? Провести группировку запросов, выделить похожие и однотипные

In [ ]:
import json

df['searchQuery'] = df[df['eventText'] == 'Выполнен поиск']['content'].apply(lambda x: json.loads(x)["docRequest"]['query'])

## этот столбец позволяет увидеть поисковый запрос пользователя

In [ ]:
queries = df[df['eventText'] == 'Выполнен поиск'].groupby('searchQuery', 
                                                as_index=False).agg({
                                                    'id': 'count'
                                                    }).sort_values(by=['id'], 
                                                    ascending=False)

queries.head(20)
##  Вывели 20 самых популярных запросов
##  Самый популярный запрос - пустой (14992)

In [ ]:
import re

## Вариант 1 (rule-based). Можно сгруппировать запросы по тому есть ли там ключевые слова "газпром", "нефть", "шельф", "разработка", "грейд" или нет

def check(query):
    if str(query) in ('газпром', 'нефть', 'шельф', 'разработка', 'грейд'):
        return 1
    else:
        return 0

    
queries['oil-related'] = queries['searchQuery'].apply(check)
 


In [ ]:
queries

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

##  Вариант 2. Кластеризация с KMeans

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(queries['searchQuery'])

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
predicted = kmeans.fit_predict(X)

In [ ]:



distribution = pd.DataFrame({'Queries': queries['searchQuery'], 
                             'Cluster': predicted}).groupby('Cluster', 
                                                            as_index=False).agg('count')
distribution


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

#  Делаем график чтобы увидеть распределение запросов по кластерам
#  Оказывается, что в распредление не нормально, 1 кластер огромный, а остальные меньше

plt.scatter(distribution['Cluster'], 
               distribution['Queries'])
plt.show()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=7)
predicted = kmeans.fit_predict(X)

In [ ]:
result_kmeans = pd.DataFrame({'Queries': queries['searchQuery'], 
              'Cluster': predicted})

In [ ]:
distribution = result_kmeans.groupby('Cluster', as_index=False).agg('count')
distribution
## как видим, результат примерно тот же самый, но кластеры "более равномерные"

In [ ]:
result_kmeans

##  смотрим таблицу, где указан кластер каждого текста

In [ ]:
result_kmeans[result_kmeans['Cluster'] == 3]

##  смотрим все запросы из 3 кластера

In [ ]:
from sklearn.cluster import DBSCAN

##  Вариант 3. Кластеризация с DBSCAN

dbscan = DBSCAN(eps=1.13, min_samples=100)
predicted = dbscan.fit_predict(X)


In [ ]:
result_dbscan = pd.DataFrame({'Queries': queries['searchQuery'], 
              'Cluster': predicted})

result_dbscan

In [ ]:
distribution = result_dbscan.groupby('Cluster', as_index=False).agg('count')
distribution

##  получается 2 больших кластера и помельче. 
##  сравнил с другими вариантами eps. 
##  Если ставить 0.5, то будет один большой кластер и куча мелких
##  Если ставить 2, т будет один большой кластер и 2 маленьких
##  C eps=1.13 тоже много мелких кластеров, но зато есть два крупных
##  Чтобы не было мелких кластеров, использовал параметр min_samples=100

Для сгруппированных запросов выделить группы запросов для которых свойственно
переход в источник (после выполнения поиска с большой вероятностью случается событие
ObjectViewedJournalEvent);

In [ ]:
##  будем использовать результат кластеризации KMeans

data = pd.DataFrame({'Queries': result_kmeans['Queries'],
                     'eventCode': df['eventCode'],
                     'cluster': result_kmeans['Cluster']}, 
                    index=result_kmeans.index)

In [ ]:

data[data['eventCode'] == 'ObjectViewedJournalEvent'].groupby('cluster', as_index=False).agg({'Queries': 'count'})


##  Как видно, кластер которому больше всего свойственно переход в источник - нулевой.

In [ ]:
import matplotlib.pyplot as plt

dist = data[data['eventCode'] == 'ObjectViewedJournalEvent'].groupby('cluster', as_index=False).agg({'Queries': 'count'})

plt.bar(dist['cluster'], dist['Queries'])

plt.xlabel('Кластеры запросов')
plt.ylabel('Кол-во переходов в источник')
plt.title('Переходы в источник для каждого из кластера')
plt.show()

Анализ текста запроса. Выделить из текста запроса: имена, аббревиатуры, опечатки,
шифры и тд

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
import spacy

In [ ]:
data.tail(30)

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
from tqdm import tqdm

tqdm.pandas()

def ner(query):
    doc = nlp(str(query))
    for ent in doc.ents:
        if ent.label_ == 'PER':  ## ищем только такие сущности, т.к. они дают имена 
            return ent.text


data['Named Entities'] = data['Queries'].progress_apply(ner)  ## ищем имена
data

In [ ]:
data[data['Named Entities'].str.contains('None')==False]  

## выводит все запросы в которых есть имена.
## у модели случаются ошибки. Например, "Мессояханефтегаз" определили как человека

In [ ]:
import re

def find_abbrevs(query):
    abbrevs = []
    tokens = str(query).split(' ')
    for token in tokens:
        abbrev = re.search('[А-Я]{2,}', token)
        if abbrev is not None:
            abbrevs.append(abbrev.group(0))


    if len(abbrevs) >= 1:
        return abbrevs
    else:
        return None


data['Abbreviations'] = data['Queries'].progress_apply(find_abbrevs)  ## ищем аббревиатуры

In [ ]:
data

In [ ]:
import re

def find_chiffre(query):
    abbrevs = []
    tokens = str(query).split(' ')
    for token in tokens:
        abbrev = re.search('\w+-[\d{2,}.]{2,}', token)
        if abbrev is not None:
            abbrevs.append(abbrev.group(0))


    if len(abbrevs) >= 1:
        return abbrevs
    else:
        return None


data['Chiffre'] = data['Queries'].progress_apply(find_chiffre)  ## Ищем шифры

In [ ]:
data